In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup

Using TensorFlow backend.


In [2]:
df=pd.read_excel('Downloads/Comment.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4825 entries, 0 to 4824
Data columns (total 2 columns):
Comment    4825 non-null object
L3         4816 non-null object
dtypes: object(2)
memory usage: 75.5+ KB


In [5]:
df1=df.dropna()

In [6]:
df1.L3.value_counts()

Overall Quality                            552
User interface                             395
Operating System/Platform/Browser          304
Performance impact                         297
PER / Product Roadmap                      289
Overall Experience                         239
Complexity                                 229
Ease of deployment/Removal                 162
Policy Configuration/management            162
Quality of documentation                   160
McAfee products                            126
Follow-up from TSE                         125
Supportability                             110
Self-help availability                      99
Detection                                   97
Ease of deployment                          97
Missing/Misleading Info                     92
3rd party                                   90
Overall Complexity to install/uninstall     73
Prevention                                  67
Number of bugs                              65
Customizabili

In [7]:
def print_plot(index):
    example = df1[df1.index == index][['Comment','L3']].values[0]
    if len(example) > 0:
        print(example[0])
        print('L3:', example[1])

In [8]:
print_plot(10)

unable to uninstall, without a lot of admin work, was easier (and quicker) to rebuild workstation OS, also to understand how your profiling work, you have to have a degree in mathematics, I still have one workstation which get the agent re-installed, even though I have have uninstalled it three times!!!
L3: Overall Complexity to install/uninstall


In [9]:
df1 = df1.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df1['Comment'] = df1['Comment'].apply(clean_text)

In [10]:
df1['Comment'] = df['Comment'].str.replace('\d+', '')

In [11]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df1['Comment'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 11397 unique tokens.


In [12]:
X = tokenizer.texts_to_sequences(df1['Comment'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (4816, 250)


In [13]:
Y = pd.get_dummies(df1['L3']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (4816, 99)


In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(4334, 250) (4334, 99)
(482, 250) (482, 99)


In [15]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(99, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

W1223 20:42:01.206908 4667901376 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:64: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1223 20:42:01.241058 4667901376 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:497: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1223 20:42:01.253402 4667901376 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3636: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1223 20:42:01.310754 4667901376 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3019: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 99)                9999      
Total params: 5,090,399
Trainable params: 5,090,399
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

W1223 20:42:15.434742 4667901376 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1223 20:42:16.373197 4667901376 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:958: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W1223 20:42:16.388770 4667901376 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:680: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 3900 samples, validate on 434 samples
Epoch 1/5
3900/3900 [==============================] - 30s 8ms/step - loss: 3.9206 - acc: 0.0956 - val_loss: 3.6095 - val_acc: 0.1152
Epoch 2/5
3900/3900 [==============================] - 26s 7ms/step - loss: 3.5676 - acc: 0.1151 - val_loss: 3.6013 - val_acc: 0.1152
Epoch 3/5
3900/3900 [==============================] - 25s 6ms/step - loss: 3.5551 - acc: 0.1149 - val_loss: 3.6090 - val_acc: 0.1152
Epoch 4/5
3900/3900 [==============================] - 25s 7ms/step - loss: 3.5307 - acc: 0.1269 - val_loss: 3.6033 - val_acc: 0.1129
Epoch 5/5
3900/3900 [==============================] - 25s 7ms/step - loss: 3.4709 - acc: 0.1451 - val_loss: 3.6209 - val_acc: 0.1060


In [17]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

482/482 [==============================] - 1s 2ms/step
Test set
  Loss: 3.574
  Accuracy: 0.093
